In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

We provide the following dataset (ASSIGNMENT_2.csv):
    
| Composition title | Composition Writers | Recording Title | Recording Writes |Action |
| --- | --- |--- | --- |--- |
| Yellow submarine | Leo Ouha |Yellow submarine(remix) |Leo Ouha |Leo Ouha |ACCEPTED |
| Shape of you | Ed Sheeran| Anaconda | Mick George | Roco Selto |Leo REJECTED |


Train an ML/DL model for pair matching of compositions and recordings.
Report and evaluate the results.

In [4]:
input_dataset = pd.read_csv("dataset/ASSIGNMENT_2.csv")

In [8]:
input_dataset.head(10)

,Composition Title,Composition Writers,Recording Title,Recording Writers,Recording Artist,Action
0,KOKAINA,YASSINE BAYBAH|DANIEL DLOUHY,Kokaina,A BAYBAH C DLOUHY,Miami Yacine,REJECTED
1,POR ESTAR CONTIGO,"MARTINEZ ESCAMILLA,FELIPE DE JESUS",Estar Contigo,MARTINEZ DE UBAGO RODRIGUEZ ALEJANDRO,"Alex, Jorge Y Lena",REJECTED
2,Gallardo (feat. Rick Ross & Yo Gotti),William Alfred / Karmin Kharbouch / Mario Mims...,Connect the Dots (feat. Yo Gotti and Rick Ross),MARIO MIMS|NIKOLAS PAPAMITROU|RICK ROSS|ROBERT...,Meek Mill,REJECTED
3,LESSON IN LEAVING,MAHER B/GOODRUM C,Lesson In Leavin',GOODRUM,Jo Dee Messina,ACCEPTED
4,QUÃ©DATE EN MIS BRAZOS QUEDATE EN MIS BRAZOS,KIKE SANTANDER,Quédate En Mis Brazos,SANTANDER KIKE,Christian Castro,ACCEPTED
5,diamonds,sia furler mikkel eriksen tor hermansen,Diamonds,BENJAMIN LEVIN/MIKKEL STORLEER ERIKSEN/SIA KAT...,Jennel Garcia,ACCEPTED
6,Can'T Take My Eyes Off Of You,Robert Gaudio|Bob Crewe,Can't Take My Eyes Off You,BOB CREWE,Cary Brothers,ACCEPTED
7,corazon,jimenez jose hernandez,"Corazon, Corazon",JIMENEZ SANDOVAL JOSE ALFREDO,Pedro Infante,REJECTED
8,CHEAP ASS WEAVE,"Belcalis ALMANZAR,Thomas Patrick BRODERICK",Cheap Ass Weave,ALMANZAR/BRODERICK/CA ALMANZAR,Cardi B.,ACCEPTED
9,BOUND 2,KANYE WEST,Bound 2,Bobby Dukes/Bobby Massey/Charlie Wilson/Che J....,Kanye West,ACCEPTED


In [11]:
input_dataset.dropna().drop_duplicates().shape


(2112, 6)